In [3]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [4]:
import expectexception

ModuleNotFoundError: No module named 'expectexception'

# Pandas

<!-- requirement: data/yelp.json.gz -->
<!-- requirement: data/PEP_2016_PEPANNRES.csv -->

In [6]:
import pandas as pd

We introduced the Pandas module and the DataFrame object in the lesson on [basic data science modules](DS_Basic_DS_Modules.ipynb). We learned how to construct a DataFrame, add data, retrieve data, and [basic reading and writing to disk](DS_IO.ipynb). Now we'll explore the DataFrame object and its powerful analysis methods in more depth.

We'll work with a data set from the online review site, Yelp. The file is stored as a compressed JSON file.

In [7]:
!ls -lh ./data/yelp.json.gz

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
import gzip
import simplejson as json

with gzip.open('./data/yelp.json.gz', 'r') as f:
    yelp_data = [json.loads(line) for line in f]

print(yelp_data[:2])
yelp_df = pd.DataFrame(yelp_data)
yelp_df.head()   



[{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA', 'full_address': '4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018', 'hours': {'Tuesday': {'close': '17:00', 'open': '08:00'}, 'Friday': {'close': '17:00', 'open': '08:00'}, 'Monday': {'close': '17:00', 'open': '08:00'}, 'Wednesday': {'close': '17:00', 'open': '08:00'}, 'Thursday': {'close': '17:00', 'open': '08:00'}}, 'open': True, 'categories': ['Doctors', 'Health & Medical'], 'city': 'Phoenix', 'review_count': 7, 'name': 'Eric Goldberg, MD', 'neighborhoods': [], 'longitude': -111.983758, 'state': 'AZ', 'stars': 3.5, 'latitude': 33.499313, 'attributes': {'By Appointment Only': True}, 'type': 'business'}, {'business_id': 'JwUE5GmEO-sH1FuwJgKBlQ', 'full_address': '6162 US Highway 51\nDe Forest, WI 53532', 'hours': {}, 'open': True, 'categories': ['Restaurants'], 'city': 'De Forest', 'review_count': 26, 'name': 'Pine Cone Restaurant', 'neighborhoods': [], 'longitude': -89.335844, 'state': 'WI', 'stars': 4.0, 'latitude': 43.238893, 'attributes

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{'Tuesday': {'close': '17:00', 'open': '08:00'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business
1,"{'Take-out': True, 'Good For': {'dessert': Fal...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],De Forest,"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business
2,"{'Take-out': True, 'Good For': {'dessert': Fal...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]",De Forest,"505 W North St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '06:00'}...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business
3,"{'Take-out': True, 'Wi-Fi': 'free', 'Takes Res...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",De Forest,"4910 County Rd V\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '10:30'}...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business
4,"{'Take-out': True, 'Has TV': False, 'Outdoor S...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]",De Forest,"631 S Main St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '11:00'}...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business


## Pandas DataFrame and Series

The Pandas DataFrame is a highly structured object. Each row corresponds with some physical entity or event. We think of all of the information in a given row as referring to one object (e.g. a business). Each column contains one type of data, both semantically (e.g. names, counts of reviews, star ratings) and syntactically.

In [11]:
yelp_df.dtypes 

attributes        object
business_id       object
categories        object
city              object
full_address      object
hours             object
latitude         float64
longitude        float64
name              object
neighborhoods     object
open                bool
review_count       int64
stars            float64
state             object
type              object
dtype: object

We can reference the columns by name, like we would with a `dict`.

In [196]:
yelp_df['city'].tail()

37933     Glendale
37934      Madison
37935      Phoenix
37936      Phoenix
37937    Edinburgh
Name: city, dtype: object

In [13]:
type(yelp_df['city'])

pandas.core.series.Series

An individual column is a Pandas `Series`. A `Series` has a `name` and a `dtype` (similar to a NumPy array). A `DataFrame` is essentially a `dict` of `Series` objects. The `Series` has an `index` attribute, which label the rows. The index is essentially a set of keys for referencing the rows. We can have an index composed of numbers, strings, timestamps, or any hashable Python object. The index will also have homogeneous type.

In [15]:
yelp_df['city'].index

RangeIndex(start=0, stop=37938, step=1)

The `DataFrame` has an `index` given by the union of indices of its constituent `Series` (we'll explore this later in more detail). Since a `DataFrame` is a `dict` of `Series`, we can select a column and then a row using square bracket notation, but not the reverse (however, the `loc` method works around this).

In [197]:
# this works
yelp_df['city'][100]

'Madison'

In [17]:
%%expect_exception KeyError

# this doesn't
yelp_df[100]['city']

UsageError: Cell magic `%%expect_exception` not found.


In [18]:
yelp_df.loc[100]

attributes       {'Take-out': True, 'Good For': {'dessert': Fal...
business_id                                 oX6KamASYXnMIe5kpVhezw
categories                                  [Chinese, Restaurants]
city                                                       Madison
full_address     4237 Lien Rd\nSte H\nMayfair Park\nMadison, WI...
hours            {'Monday': {'close': '21:30', 'open': '09:00'}...
latitude                                                   43.1206
longitude                                                 -89.3084
name                                                   Flaming Wok
neighborhoods                                       [Mayfair Park]
open                                                          True
review_count                                                    20
stars                                                          3.5
state                                                           WI
type                                                      busi

Understanding the underlying structure of the `DataFrame` object as a `dict` of `Series` will help you avoid errors and help you think about how the `DataFrame` should behave when we begin doing more complicated analysis.

We can _aggregate_ data in a `DataFrame` using methods like `mean`, `sum`, `count`, and `std`. To view a collection of summary statistics for each column we can use the `describe` method.

In [19]:
yelp_df.describe()

,latitude,longitude,review_count,stars
count,37938.000000,37938.000000,37938.000000,37938.000000
mean,36.530288,-104.097172,29.300648,3.672914
std,5.822588,28.365375,92.893064,0.888730
min,32.876638,-115.369725,3.000000,1.000000
25%,33.476739,-115.136389,4.000000,3.000000
50%,33.680247,-112.068820,8.000000,3.500000
75%,36.141875,-111.883274,21.000000,4.500000
max,56.033777,-3.050088,4084.000000,5.000000


The utility of a DataFrame comes from its ability to split data into groups, using the `groupby` method, and then perform custom aggregations using the `apply` or `aggregate` method. This process of splitting the data into groups, applying an aggregation, and then collecting the results is [discussed in detail in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html), and is one of the main focuses of this notebook.

## DataFrame construction

Since a `DataFrame` is a `dict` of `Series`, the natural way to construct a `DataFrame` is to use a `dict` of `Series`-like objects.

In [20]:
s = pd.Series(['a','b','c','d','f','g'],[1, 2, 3,8,9,9])

print(s.index)



Int64Index([1, 2, 3, 8, 9, 9], dtype='int64')


In [21]:
from string import ascii_letters, digits
import numpy as np
import datetime

In [22]:
usernames = ['alice36', 'bob_smith', 'eve']

passwords = [''.join(np.random.choice(list(ascii_letters + digits), 8)) for x in range(3)]
creation_dates = [datetime.datetime.now().date() - datetime.timedelta(int(x)) for x in np.random.randint(0, 1500, 3)]
# ls = [int(x) for x in np.random.randint(0, 1500, 5)]


In [23]:
df = pd.DataFrame({'username': usernames, 'password': passwords, 'date-created': pd.to_datetime(creation_dates)})
df

,username,password,date-created
0,alice36,TwUAcgbp,2017-09-04
1,bob_smith,U4vdpCMQ,2017-09-29
2,eve,qTvRfn4f,2016-05-21


In [24]:
df.dtypes

username                object
password                object
date-created    datetime64[ns]
dtype: object

The `DataFrame` is also closely related to the NumPy `ndarray`.

In [26]:
random_data = np.random.random((4,3))
random_data

array([[0.49835457, 0.71103272, 0.01250212],
       [0.02096874, 0.15578555, 0.84064346],
       [0.5496386 , 0.65097083, 0.45054964],
       [0.25885943, 0.16595915, 0.81770324]])

In [30]:
df_random = pd.DataFrame(random_data, columns=['a', 'b', 'c'] ,index=[2,3,56,7])
df_random.index=index=[0,1,2,7]
df_random

,a,b,c
0,0.498355,0.711033,0.012502
1,0.020969,0.155786,0.840643
2,0.549639,0.650971,0.450550
7,0.258859,0.165959,0.817703


To add a new column or row, we simply use `dict`-like assignment.

In [31]:
emails = ['alice.chan@gmail.com', 'bwsmith1983@gmail.com', 'fakemail123@yahoo.com']
df['email'] = emails
df

,username,password,date-created,email
0,alice36,TwUAcgbp,2017-09-04,alice.chan@gmail.com
1,bob_smith,U4vdpCMQ,2017-09-29,bwsmith1983@gmail.com
2,eve,qTvRfn4f,2016-05-21,fakemail123@yahoo.com


In [32]:
# loc references index value, NOT position
# for position use iloc
df.loc[7] = ['2015-01-29', '38uzFJ1n', 'melvintherobot', 'moviesrgood@moviesrgood.com']
df

,username,password,date-created,email
0,alice36,TwUAcgbp,2017-09-04 00:00:00,alice.chan@gmail.com
1,bob_smith,U4vdpCMQ,2017-09-29 00:00:00,bwsmith1983@gmail.com
2,eve,qTvRfn4f,2016-05-21 00:00:00,fakemail123@yahoo.com
7,2015-01-29,38uzFJ1n,melvintherobot,moviesrgood@moviesrgood.com


We can also drop columns and rows.

In [33]:
df.drop(7)


,username,password,date-created,email
0,alice36,TwUAcgbp,2017-09-04 00:00:00,alice.chan@gmail.com
1,bob_smith,U4vdpCMQ,2017-09-29 00:00:00,bwsmith1983@gmail.com
2,eve,qTvRfn4f,2016-05-21 00:00:00,fakemail123@yahoo.com


In [34]:
df

,username,password,date-created,email
0,alice36,TwUAcgbp,2017-09-04 00:00:00,alice.chan@gmail.com
1,bob_smith,U4vdpCMQ,2017-09-29 00:00:00,bwsmith1983@gmail.com
2,eve,qTvRfn4f,2016-05-21 00:00:00,fakemail123@yahoo.com
7,2015-01-29,38uzFJ1n,melvintherobot,moviesrgood@moviesrgood.com


In [35]:
# to drop a column, need axis=1
df.drop('email', axis=1, inplace=True)
df

,username,password,date-created
0,alice36,TwUAcgbp,2017-09-04 00:00:00
1,bob_smith,U4vdpCMQ,2017-09-29 00:00:00
2,eve,qTvRfn4f,2016-05-21 00:00:00
7,2015-01-29,38uzFJ1n,melvintherobot


Notice when we dropped the `'email'` column, the row at index 3 was in the `DataFrame`, even though we just dropped it! Most operations in Pandas return a _copy_ of the `DataFrame`, rather than modifying the `DataFrame` object itself. Therefore, in order to permanently alter the `DataFrame`, we either need to reassign the `df` variable, or use the `inplace` keyword.

In [36]:
df.drop('a', inplace=True)
df

KeyError: "['a'] not found in axis"

Since the `index` and column names are important for interacting with data in the DataFrame, we should make sure to set them to useful values. We can do this during construction or after.

In [37]:
df = pd.DataFrame({'email': emails, 'password': passwords, 'date-created': creation_dates}, index=usernames)
df.index.name = 'users' # it can be helpful to give the index a name
df

,email,password,date-created
users,,,
alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


In [38]:
# alternatively
s=list(zip(usernames, emails, passwords, creation_dates))
print(s)
df = pd.DataFrame(list(zip(usernames, emails, passwords, creation_dates)),columns=['a', 'b', 'c','d'])
df

[('alice36', 'alice.chan@gmail.com', 'TwUAcgbp', datetime.date(2017, 9, 4)), ('bob_smith', 'bwsmith1983@gmail.com', 'U4vdpCMQ', datetime.date(2017, 9, 29)), ('eve', 'fakemail123@yahoo.com', 'qTvRfn4f', datetime.date(2016, 5, 21))]


,a,b,c,d
0,alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
1,bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
2,eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


In [39]:
df.columns = ['username', 'email', 'password', 'date-created']
df.set_index('username', inplace=True)
df


,email,password,date-created
username,,,
alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


In [41]:
# to reset index to a column
df.reset_index(inplace=True)
sorted(df.index)

[0, 1, 2]

We can have multiple levels to an index. We'll discover that for some data sets it is necessary to have multiple levels to the index in order to uniquely identify a row.

In [42]:
df.set_index(['username', 'email'])

,,password,date-created
username,email,,
alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


### Reading data from file

We can also construct a DataFrame using data stored in a file or received from a website. The data source might be [JSON](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html), [HTML](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html), [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv), [Excel](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html), [Python pickle](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_pickle.html), or even a [database connection](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html). Each format will have its own methods for reading and writing data that take different arguments. The arguments of these methods usually depend on the particular formatting of the file. For example, the values in a CSV might be separate by commas or semi-colons, it might have a header or it might not.

The `read_csv` method has to deal with the most formatting possibilities, so we will explore that method with a few examples. Try to apply these ideas when working with other file formats, but keep in mind that each format and read method is different. Always check [the Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/io.html) when having trouble with reading or writing data.

In [43]:
csv = [','.join(map(lambda x: str(x), row)) for row in np.vstack([df.columns, df])]
with open('./data/ppe.csv', 'w') as f:
    [f.write(line + '\n') for line in csv]

!cat ./data/read_csv_example.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
sorted(pd.read_csv('./data/read_csv_example.csv'))

['date-created', 'email', 'password', 'username']

In [45]:
# we can also set an index from the data
pd.read_csv('./data/read_csv_example.csv', index_col=0)

,email,password,date-created
username,,,
alice36,alice.chan@gmail.com,VhipAl4D,2017-06-02
bob_smith,bwsmith1983@gmail.com,bGlGjvP8,2019-04-29
eve,fakemail123@yahoo.com,8jFJpNVv,2016-08-02


In [46]:
# what if our data had no header?
with open('./data/read_csv_noheader_example.csv', 'w') as f:
    [f.write(line + '\n') for i, line in enumerate(csv) if i != 0]
    


In [47]:

pd.read_csv('./data/read_csv_noheader_example.csv', names=['username', 'email', 'password', 'date-created'], header=None)

,username,email,password,date-created
0,alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
1,bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
2,eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


In [49]:
# what if our data was tab-delimited?
tsv = ['\t'.join(map(lambda x: str(x), row)) for row in np.vstack([df.columns, df])]
with open('./data/read_csv_example.tsv', 'w') as f:
    [f.write(line + '\n') for line in tsv]



In [51]:
pd.read_csv('./data/read_csv_example.tsv', delimiter='\t')

,username,email,password,date-created
0,alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
1,bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
2,eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


Even within a single file format, data can be arranged and formatted in many ways. These have been just a few examples of the kinds of arguments you might need to use with `read_csv` in order to read data into a DataFrame in an organized way.

## Filtering DataFrames

One of the powerful analytical tools of the Pandas DataFrame is its syntax for filtering data. Often we'll only want to work with a certain subset of our data based on some criteria. Let's look at our Yelp data for an example.

In [55]:
yelp_df.head()
yelp_df['state'].unique()

array(['AZ', 'WI', 'NV', 'NC', 'GA', 'CA', 'ON', 'EDH', 'MLN', 'ELN',
       'SCB', 'XGL', 'NTH', 'NY', 'FIF', 'KHL', 'MN', 'MA'], dtype=object)

We see the Yelp data set has a `'state'` column. If we are only interested in businesses in Arizona (AZ), we can filter the DataFrame and select only that data.

In [57]:
az_yelp_df = yelp_df[yelp_df['state'] == 'AZ']
az_yelp_df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,{'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{'Tuesday': {'close': '17:00', 'open': '08:00'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business
297,"{'Take-out': True, 'Wi-Fi': 'no', 'Alcohol': '...",x5Mv61CnZLohZWxfCVCPTQ,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004",{},33.479542,-112.073418,Domino's Pizza,[],True,12,2.5,AZ,business
298,"{'Take-out': True, 'Noise Level': 'quiet', 'De...",2ZnCITVa0abGce4gZ6RhIw,"[American (New), Sandwiches, Restaurants]",Phoenix,"1850 N Central Ave\nPhoenix, AZ 85004",{},33.468988,-112.074315,Viad Tower Restaurants,[],True,5,3.5,AZ,business
299,"{'Alcohol': 'full_bar', 'Price Range': 1, 'Noi...",EmzaQR5hQlF0WIl24NxAZA,"[American (New), Nightlife, Dance Clubs, Resta...",Phoenix,"132 E Washington St\nPhoenix, AZ 85004","{'Sunday': {'close': '02:00', 'open': '21:00'}...",33.448399,-112.071702,Sky Lounge,[],True,20,2.5,AZ,business
300,"{'Price Range': 2, 'Alcohol': 'full_bar', 'Goo...",SiwN7f0N4bs4ZtPc4yPgiA,"[Nightlife, Dance Clubs]",Phoenix,"710 N Central Ave\nPhoenix, AZ 85004",{},33.456068,-112.074225,Palazzo,[],True,15,2.5,AZ,business


In [62]:
az_yelp_df['state'].index

Int64Index([    0,   297,   298,   299,   300,   301,   302,   303,   304,
              305,
            ...
            37914, 37916, 37917, 37918, 37920, 37921, 37923, 37933, 37935,
            37936],
           dtype='int64', length=19927)

We can combine criteria using logic. What if we're only interested in businesses with more than 10 reviews in Arizona?

In [63]:
yelp_df[(yelp_df['state'] == 'AZ') & (yelp_df['review_count'] > 10)].head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
297,"{'Take-out': True, 'Wi-Fi': 'no', 'Alcohol': '...",x5Mv61CnZLohZWxfCVCPTQ,"[Sandwiches, Pizza, Chicken Wings, Restaurants]",Phoenix,"2819 N Central Ave\nPhoenix, AZ 85004",{},33.479542,-112.073418,Domino's Pizza,[],True,12,2.5,AZ,business
299,"{'Alcohol': 'full_bar', 'Price Range': 1, 'Noi...",EmzaQR5hQlF0WIl24NxAZA,"[American (New), Nightlife, Dance Clubs, Resta...",Phoenix,"132 E Washington St\nPhoenix, AZ 85004","{'Sunday': {'close': '02:00', 'open': '21:00'}...",33.448399,-112.071702,Sky Lounge,[],True,20,2.5,AZ,business
300,"{'Price Range': 2, 'Alcohol': 'full_bar', 'Goo...",SiwN7f0N4bs4ZtPc4yPgiA,"[Nightlife, Dance Clubs]",Phoenix,"710 N Central Ave\nPhoenix, AZ 85004",{},33.456068,-112.074225,Palazzo,[],True,15,2.5,AZ,business
301,{},M-MFa0YErVBbFW2IJhGJmw,"[Auto Repair, Automotive, Tires, Oil Change St...",Phoenix,"386 E Virginia Ave\nPhoenix, AZ 85004","{'Tuesday': {'close': '18:00', 'open': '07:30'...",33.476784,-112.065806,Virginia Auto Service,[],True,28,4.0,AZ,business
302,"{'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...",KPoTixdjoJxSqRSEApSAGg,"[Thai, Restaurants]",Phoenix,"2631 N Central Ave\nPhoenix, AZ 85004","{'Monday': {'close': '21:00', 'open': '11:00'}...",33.477934,-112.073524,Wild Thaiger,[],True,281,4.0,AZ,business


How does this filtering work?

When we write `yelp_df['state'] == 'AZ'`, Pandas selects the `'state'` column and checks whether each row is `'AZ'`. If so, that row is marked `True`, and if not, it is marked `False`. This is how we would normally expect a conditional to work, only now applied to an entire Pandas `Series`. We end up with a Pandas `Series` of Boolean variables.

In [ ]:
yelp_data.count

In [50]:
(yelp_df['state'] == 'AZ').head()

0     True
1    False
2    False
3    False
4    False
Name: state, dtype: bool

We can use a `Series` (or any similar object) of Boolean variables to index the DataFrame.

In [52]:
df

,username,email,password,date-created
0,alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
1,bob_smith,bwsmith1983@gmail.com,U4vdpCMQ,2017-09-29
2,eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


In [53]:
df[[True, False, True]]

,username,email,password,date-created
0,alice36,alice.chan@gmail.com,TwUAcgbp,2017-09-04
2,eve,fakemail123@yahoo.com,qTvRfn4f,2016-05-21


This let's us filter a DataFrame using idiomatic logical expressions like `yelp_df['review_count'] > 10`.

As another example, let's consider the `'open'` column, which is a `True`/`False` flag for whether a business is open. This is also a Boolean Pandas `Series`, so we can just use it directly.

In [135]:
# the open businesses
yelp_df[yelp_df['open']].head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,delivery
0,{'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]",Phoenix,"4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{'Tuesday': {'close': '17:00', 'open': '08:00'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business,None
1,"{'Take-out': True, 'Good For': {'dessert': Fal...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],De Forest,"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business,False
2,"{'Take-out': True, 'Good For': {'dessert': Fal...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]",De Forest,"505 W North St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '06:00'}...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business,False
3,"{'Take-out': True, 'Wi-Fi': 'free', 'Takes Res...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...",De Forest,"4910 County Rd V\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '10:30'}...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business,False
4,"{'Take-out': True, 'Has TV': False, 'Outdoor S...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]",De Forest,"631 S Main St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '11:00'}...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business,None


In [ ]:
# the closed businesses
yelp_df[~yelp_df['open']].head()

Notice in an earlier expression we wrote `(yelp_df['state'] == 'AZ') & (yelp_df['review_count'] > 10)`. Normally in Python we use the word `and` when we are working with logic. In Pandas we have to use _bit-wise_ logical operators; all that's important to know is the following equivalencies:

`~` = `not`  
`&` = `and`  
`|` = `or`  

We can also use Panda's built-in [string operations](https://pandas.pydata.org/pandas-docs/stable/text.html) for doing pattern matching. For example, there are a lot of businesses in Las Vegas in our data set. However, there are also businesses in 'Las Vegas East' and 'South Las Vegas'. To get all of the Las Vegas businesses I might do the following.

In [136]:
vegas_yelp_df = yelp_df[yelp_df['city'].str.contains('Vegas')]
vegas_yelp_df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,delivery
4348,"{'Parking': {'garage': False, 'street': False,...",uzztTCsg3XAKlYqmLO40aw,"[Arts & Entertainment, Shopping, Books, Mags, ...",Las Vegas,"5835 S Eastern Ave\nSoutheast\nLas Vegas, NV 8...","{'Monday': {'close': '21:00', 'open': '10:00'}...",36.083936,-115.119422,Psychic Eye Book Shops,[Southeast],True,7,4.5,NV,business,None
4360,"{'Parking': {'garage': True, 'street': False, ...",IspFZDcpu9W5NvvJ7QgJZg,"[Women's Clothing, Men's Clothing, Fashion, Sh...",Las Vegas,The Forum Shops At Caesars\n3500 Las Vegas Blv...,"{'Monday': {'close': '21:00', 'open': '10:00'}...",36.124423,-115.146372,Gap,[Eastside],True,6,4.5,NV,business,None
4364,"{'By Appointment Only': False, 'Parking': {'ga...",Dvulg9cc3XHkt5DkBUekhw,"[Hair Salons, Beauty & Spas]",Las Vegas,467 E Silverado Ranch Blvd\nSte 175\nSoutheast...,{},36.012150,-115.153303,Great Clips,[Southeast],True,10,2.5,NV,business,None
4441,"{'Alcohol': 'full_bar', 'Music': {'video': Fal...",_SM8UKIwBNbmj1r629ipoQ,"[Wine Bars, Bars, Restaurants, Nightlife, Ital...",Las Vegas,"1916 Village Center Cir\nSummerlin\nLas Vegas,...","{'Monday': {'close': '23:00', 'open': '11:00'}...",36.192414,-115.304396,Chianti Cafe,[Summerlin],False,9,3.5,NV,business,False
4489,{},Zaw0IAXwopKX4MV23MHQRw,"[Car Wash, Automotive]",Las Vegas,"2718 E Tropicana Avenue\nEastside\nLas Vegas, ...",{},36.100507,-115.113492,Terrible Herbst,"[Eastside, Southeast]",True,3,1.5,NV,business,None


In [137]:
vegas_yelp_df['city'].unique()

array(['Las Vegas', 'North Las Vegas', 'N Las Vegas', 'N. Las Vegas',
       'C Las Vegas', 'Las Vegas ', 'Las Vegas, NV 89147',
       'Las Vegas East', 'North Las Vegas ', 'N E Las Vegas',
       'N W Las Vegas', 'Lake Las Vegas', 'South Las Vegas'], dtype=object)

## Applying functions and data aggregation

To analyze the data in the dataframe, we'll need to be able to apply functions to it. Pandas has many mathematical functions built in already, and DataFrames and Series can be passed to NumPy functions (since they behave like NumPy arrays).

In [100]:
log_review_count = np.log(yelp_df['review_count'])
print(log_review_count.head())
print(log_review_count.shape)
print(yelp_df.head())

0    1.945910
1    3.258097
2    2.772589
3    1.945910
4    1.098612
Name: review_count, dtype: float64
(37938,)
                                          attributes             business_id  \
0                      {'By Appointment Only': True}  vcNAWiLM4dR7D2nwwJ7nCA   
1  {'Take-out': True, 'Good For': {'dessert': Fal...  JwUE5GmEO-sH1FuwJgKBlQ   
2  {'Take-out': True, 'Good For': {'dessert': Fal...  uGykseHzyS5xAMWoN6YUqA   
3  {'Take-out': True, 'Wi-Fi': 'free', 'Takes Res...  LRKJF43s9-3jG9Lgx4zODg   
4  {'Take-out': True, 'Has TV': False, 'Outdoor S...  RgDg-k9S5YD_BaxMckifkg   

                                          categories       city  \
0                        [Doctors, Health & Medical]    Phoenix   
1                                      [Restaurants]  De Forest   
2              [American (Traditional), Restaurants]  De Forest   
3  [Food, Ice Cream & Frozen Yogurt, Fast Food, R...  De Forest   
4                             [Chinese, Restaurants]  De Forest   

  

In [94]:
mean_review_count = yelp_df['review_count'].mean()
print(mean_review_count)

29.300648426379883


In the first example we took the _logarithm_ of the review count for each business. In the second case, we calculated the mean review count of all businesses. In the first case, we ended up with a number for each business. We _transformed_ the review counts using the logarithm. In the second case, we _summarized_ the review counts of all the businesses in one number. This summary is a form of _data aggregation_, in which we take many data points and combine them into some smaller representation. The functions we apply to our data sets will either be in the category of **transformations** or **aggregations**.

Sometimes we will need to transform our data in order for it to be usable. For instance, in the `'attributes'` column of our DataFrame, we have a `dict` for each business listing all of its properties. If I wanted to find a restaurant that offers delivery service, it would be difficult for me to filter the DataFrame, even though that information is in the `'attributes'` column. First, I need to transform the `dict` into something more useful.

In [101]:
def get_delivery_attr(attr_dict):
    return attr_dict.get('Delivery')

If we give this function a `dict` from the `'attributes'` column, it will look for the `'Delivery'` key. If it finds that key, it returns the value. If it doesn't find the key, it will return none.

In [109]:
print(yelp_df.loc[0, 'attributes'])
print(get_delivery_attr(yelp_df.loc[1, 'attributes']))
print(get_delivery_attr(yelp_df.loc[2, 'attributes']))
print(yelp_df['attributes'])
print(yelp_df.attributes.loc[0])

{'By Appointment Only': True}
False
False
0                            {'By Appointment Only': True}
1        {'Take-out': True, 'Good For': {'dessert': Fal...
2        {'Take-out': True, 'Good For': {'dessert': Fal...
3        {'Take-out': True, 'Wi-Fi': 'free', 'Takes Res...
4        {'Take-out': True, 'Has TV': False, 'Outdoor S...
5                                                       {}
6                                                       {}
7                                        {'Wi-Fi': 'free'}
8                                                       {}
9        {'Alcohol': 'full_bar', 'Noise Level': 'loud',...
10       {'Accepts Credit Cards': True, 'Wi-Fi': 'free'...
11                                 {'Good for Kids': True}
12       {'Take-out': True, 'Good For': {'dessert': Fal...
13       {'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...
14       {'Take-out': True, 'Caters': True, 'Attire': '...
15       {'Take-out': True, 'Good For': {'dessert': Fal...
16       {'Tak

We could iterate over the rows of `yelp_df['attributes']` to get all of the values, but there is a better way. DataFrames and Series have an `apply` method that allows us to apply our function to the entire data set at once, like we did earlier with `np.log`.

In [108]:
delivery_attr = yelp_df['attributes'].apply(get_delivery_attr)
delivery_attr.head()

0     None
1    False
2    False
3    False
4     None
Name: attributes, dtype: object

We can make a new column in our DataFrame with this transformed (and useful) information.

In [127]:
yelp_df['delivery'] = delivery_attr

# to find businesses that deliver
yelp_df[yelp_df['delivery'].fillna(False)].count


<bound method DataFrame.count of                                               attributes  \
21     {'Take-out': True, 'Takes Reservations': False...   
64     {'Take-out': True, 'Alcohol': 'none', 'Takes R...   
75     {'Take-out': True, 'Good For': {'dessert': Fal...   
79     {'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...   
82     {'Delivery': True, 'Waiter Service': True, 'Ta...   
100    {'Take-out': True, 'Good For': {'dessert': Fal...   
106    {'BYOB': False, 'Take-out': True, 'Wi-Fi': 'no...   
113    {'Take-out': True, 'Wi-Fi': 'free', 'Good For'...   
120    {'Take-out': True, 'Good For': {'dessert': Fal...   
127    {'Take-out': True, 'Alcohol': 'none', 'Deliver...   
168    {'Take-out': True, 'Good For': {'dessert': Fal...   
180    {'Delivery': True, 'Takes Reservations': False...   
190    {'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...   
211    {'Delivery': True, 'Good For': {'dessert': Fal...   
231    {'Take-out': True, 'Accepts Credit Cards': Tru...   
255    

It's less common (though possible) to use `apply` on an entire DataFrame rather than just one column. Since a DataFrame might contain many types of data, we won't usually want to apply the same transformation or aggregation across all of the columns.

## Data aggregation with `groupby`

Data aggregation is an [_overloaded_](https://en.wikipedia.org/wiki/Function_overloading) term. It refers to both data summarization (as above) but also to the combining of different data sets.

With our Yelp data, we might be interested in comparing the star ratings of businesses in different cities. We could calculate the mean star rating for each city, and this would allow us to easily compare them. First we would have to split up our data by city, calculate the mean for each city, and then combine it back at the end. This procedure is known as [split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/groupby.html) and is a classic example of data aggregation (in the sense of both summarizing data and also combining different data sets).

We achieve the splitting and recombining using the `groupby` method.

In [111]:
stars_by_city = yelp_df.groupby('city')['stars'].mean()
stars_by_city.head()

city
Ahwatukee          3.687500
Anthem             3.781818
Apache Junction    3.637500
Arcadia            5.000000
Atlanta            3.500000
Name: stars, dtype: float64

We can also apply multiple functions at once. It might be helpful to know the standard deviation of star ratings, the total number of reviews, and the count of businesses as well.

In [83]:
agg_by_city = yelp_df.groupby('city').agg({'stars': {'mean': 'mean', 'std': 'std'}, 'review_count': 'sum', 'business_id': 'count'})
agg_by_city.head()

C:\Users\eyyup.direk\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


stars           review_count business_id
                     mean       std          sum       count
city                                                        
Ahwatukee        3.687500  1.066955          258           8
Anthem           3.781818  0.756231          727          55
Apache Junction  3.637500  1.028010          853          80
Arcadia          5.000000       NaN            4           1
Atlanta          3.500000       NaN           16           1

In [84]:
agg_by_city['stars']['mean'][:10]

city
Ahwatukee            3.687500
Anthem               3.781818
Apache Junction      3.637500
Arcadia              5.000000
Atlanta              3.500000
Avondale             3.538627
Black Canyon City    3.000000
Bonnyrigg            3.750000
Boulder City         4.136364
Buckeye              3.408451
Name: mean, dtype: float64

In [91]:
l=[0,7,8,8,9,7,8,8,9]
max(set(l),key=l.count)

8

In [85]:
# unstacking the columns
print(agg_by_city.columns.get_level_values)
new_columns = map(lambda x: '_'.join(x),
                  zip(agg_by_city.columns.get_level_values(0),
                      agg_by_city.columns.get_level_values(1)))
agg_by_city.columns = new_columns
agg_by_city.head()

<bound method MultiIndex.get_level_values of MultiIndex(levels=[['stars', 'review_count', 'business_id'], ['count', 'mean', 'std', 'sum']],
           labels=[[0, 0, 1, 2], [1, 2, 3, 0]])>


,stars_mean,stars_std,review_count_sum,business_id_count
city,,,,
Ahwatukee,3.687500,1.066955,258,8
Anthem,3.781818,0.756231,727,55
Apache Junction,3.637500,1.028010,853,80
Arcadia,5.000000,NaN,4,1
Atlanta,3.500000,NaN,16,1


How does this work? What does `groupby` do? Let's start by inspecting the result of `groupby`.

In [64]:
by_city = yelp_df.groupby('city')
print(by_city)

In [115]:
dir(by_city)

['__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_block_agg_axis',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_decide_output_index',
 '_def_str',
 '_define_paths',
 '_deprecations',
 '_dir_additio

In [66]:
print(type(by_city.groups))
list(by_city.groups.items())[:5]

<class 'dict'>


[('Ahwatukee',
  Int64Index([1718, 1740, 13572, 17787, 20361, 22856, 28608, 30499], dtype='int64')),
 ('Anthem',
  Int64Index([ 1946,  1948,  1949,  1950,  1951,  1952,  1953,  1954,  1955,
               1956,  1957,  1958,  1961,  1962, 11455, 12008, 12572, 12870,
              13007, 13069, 13286, 13500, 13636, 13703, 13744, 13827, 13935,
              15483, 15581, 15756, 19358, 21295, 21303, 21327, 22354, 22428,
              22654, 22782, 23215, 24937, 25026, 26116, 26300, 26331, 26841,
              26937, 30039, 30170, 31565, 32525, 33027, 33492, 33983, 34663,
              35433],
             dtype='int64')),
 ('Apache Junction',
  Int64Index([ 2392,  2393,  2394,  2395,  2396,  2397,  2398,  2399,  2400,
               2401,  2402,  2403,  2404,  2406,  2407,  2408,  2409,  2410,
               2412,  2413,  2414,  2415,  2417,  2418,  2419,  7305,  7524,
               7525,  7528,  7529,  7530,  7531,  7532,  7533,  7534, 10813,
              11315, 11660, 12662, 12858, 12

In [121]:
by_city.get_group('Anthem').head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,delivery
1946,"{'Parking': {'garage': False, 'street': False,...",Y0Ife2OJPs5QSL-7dyMF5Q,"[Fashion, Shopping, Shoe Stores]",Anthem,"4250 W Anthem Way\nSte 375\nAnthem, AZ 85086",{},33.874003,-112.150162,Nike Factory Store,[],True,8,4.0,AZ,business,None
1948,"{'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...",v1ocqAm5soWTNPttLQhSZw,"[Chinese, Restaurants]",Anthem,"3434 W Anthem Way\nSte 118\nAnthem, AZ 85086",{},33.864429,-112.135305,Shanghai Club,[],True,50,3.5,AZ,business,False
1949,"{'Parking': {'garage': False, 'street': False,...",HMDymN9TbFUln9AkqHMpVw,"[Department Stores, Fashion, Shopping]",Anthem,"4250 W Anthem Way\nAnthem, AZ 85086",{},33.875410,-112.151046,Polo Ralph Lauren Factory Store,[],True,3,3.0,AZ,business,None
1950,"{'Parking': {'garage': False, 'street': False,...",XJv7dUb5fMRaZAWVZjhtvw,"[Shopping, Outlet Stores]",Anthem,"4250 W Anthem Way\nAnthem, AZ 85086",{},33.873311,-112.149875,Outlets At Anthem,[],True,64,3.5,AZ,business,None
1951,"{'Ambience': {'romantic': False, 'intimate': F...",xPDbCj9tfp5nnGzYzoCHwA,"[Sandwiches, Restaurants]",Anthem,"3655 W Anthem Way\nSte C105\nAnthem, AZ 85086",{},33.864913,-112.138854,Subway,[],True,6,3.0,AZ,business,None


When we use `groupby` on a column, Pandas builds a `dict`, using the unique elements of the column as the keys and the index of the rows in each group as the values. This `dict` is stored in the `groups` attribute. Pandas can then use this `dict` to direct the application of aggregating functions over the different groups.

## Sorting

Even though the DataFrame in many ways behaves similarly to a `dict`, it also is ordered. Therefore we can sort the data in it. Pandas provides two sorting methods, `sort_values` and `sort_index`.

In [98]:
yelp_df.sort_values('latitude').head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
2455,"{'Take-out': True, 'Alcohol': 'beer_and_wine',...",2gBKQ-epOTPMNvXoGyyVbQ,"[Buffets, Chinese, Restaurants]",Casa Grande,"1275 E Florence Blvd\nSte 5\nCasa Grande, AZ 8...",{},32.876638,-111.747780,China King,[],True,4,3.0,AZ,business
31355,"{'Good For': {'dessert': False, 'latenight': F...",24g1ECQgdTw3BHokYCFWZA,"[Steakhouses, Restaurants]",Casa Grande,"109 E 2nd St\nCasa Grande, AZ 85122",{},32.876848,-111.753461,Casa Grande Steakhouse,[],False,4,2.5,AZ,business
35454,"{'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...",TiO1u_VYeujfFSldjWvIjw,"[Cuban, Restaurants]",Casa Grande,"109 E 2nd St\nCasa Grande, AZ 85122","{'Tuesday': {'close': '21:00', 'open': '11:00'...",32.876866,-111.753341,Feli's Cuban Kitchen,[],True,24,4.5,AZ,business
2454,"{'Take-out': True, 'Good For': {'dessert': Fal...",T9BPnYNKtk6B42W0cJJXnA,"[Breakfast & Brunch, Sandwiches, Restaurants]",Casa Grande,"100 W 2nd St\nCasa Grande, AZ 85222","{'Monday': {'close': '15:00', 'open': '07:00'}...",32.877381,-111.753670,Cook E Jar Bakery & Cafe,[],True,21,4.0,AZ,business
2437,{},7AW8xNS5Meaxw4u5PRtYTw,"[Shopping, Guns & Ammo]",Casa Grande,"104 W 2nd St\nCasa Grande, AZ 85222",{},32.877524,-111.753795,Buck's Guns,[],True,3,1.0,AZ,business


In [ ]:
yelp_df.set_index('business_id').sort_index().head()

Don't forget that most Pandas operations return a copy of the DataFrame, and do not update the DataFrame in place (unless we tell it to)!

## Joining data sets

Often we will want to augment one data set with data from another. For instance, businesses in big cities probably get more reviews than those in small cities. It could be useful to scale the review counts by the city's population. To do that, we'll need to add population data to the Yelp data. We can get population data from the US census.

In [93]:
census = pd.read_csv('./data/PEP_2016_PEPANNRES.csv', skiprows=[1])

census.head()

,GEO.id,GEO.id2,GEO.display-label,rescen42010,resbase42010,respop72010,respop72011,respop72012,respop72013,respop72014,respop72015,respop72016
0,0610000US0900104720,900104720,"Bethel town, Fairfield County, Connecticut",18584.0,18584,18642,18942,19150,19267,19316,19463,19627
1,0610000US0900108070,900108070,"Bridgeport town, Fairfield County, Connecticut",144229.0,144236,144912,146226,146962,147666,147560,147090,145936
2,0610000US0900108980,900108980,"Brookfield town, Fairfield County, Connecticut",16452.0,16452,16483,16635,16785,16859,17016,17087,17098
3,0610000US0900118500,900118500,"Danbury town, Fairfield County, Connecticut",80893.0,80903,81356,82182,82788,83678,83687,84236,84992
4,0610000US0900118850,900118850,"Darien town, Fairfield County, Connecticut",20732.0,20732,20777,20975,21127,21342,21641,21718,21744


In [99]:
# construct city & state fields
census['city'] = census['GEO.display-label'].apply(lambda x: x.split(', ')[0])
census['state'] = census['GEO.display-label'].apply(lambda x: x.split(', ')[2])

In [100]:
# convert state names to abbreviations

print(census['state'].unique())

['Connecticut' 'Illinois' 'Indiana' 'Kansas' 'Maine' 'Massachusetts'
 'Michigan' 'Minnesota' 'Missouri' 'Nebraska' 'New Hampshire' 'New Jersey'
 'New York' 'North Dakota' 'Ohio' 'Pennsylvania' 'Rhode Island'
 'South Dakota' 'Vermont' 'Wisconsin']


In [101]:
state_abbr = dict(zip(census['state'].unique(), ['CT', 'IL', 'IN', 'KS', 'ME', 'MA', 'MI', 'MN', 'MO', 'NE', 'NH', 'NJ', 'NY', 'ND', 'OH', 'PA', 'RI', 'SD', 'VT', 'WI']))

In [102]:
census['state'] = census['state'].replace(state_abbr)

In [103]:
# remove last word (e.g. 'city', 'town', township', 'borough', 'village') from city names

census['city'] = census['city'].apply(lambda x: ' '.join(x.split(' ')[:-1]))

In [104]:
merged_df = yelp_df.merge(census, on=['state', 'city'])
merged_df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,...,GEO.display-label,rescen42010,resbase42010,respop72010,respop72011,respop72012,respop72013,respop72014,respop72015,respop72016
0,"{'Take-out': True, 'Caters': True, 'Attire': '...",HxPpZSY6Q1eARuiahhra6A,"[Event Planning & Services, Party & Event Plan...",Middleton,"6401 University Ave\nMiddleton, WI 53562",{},43.093265,-89.491800,Crandalls Carryout & Catering,[],...,"Middleton city, Dane County, Wisconsin",17442.0,17558,17622,17903,18271,18522,18694,18955,19109
1,"{'Take-out': True, 'Caters': True, 'Attire': '...",HxPpZSY6Q1eARuiahhra6A,"[Event Planning & Services, Party & Event Plan...",Middleton,"6401 University Ave\nMiddleton, WI 53562",{},43.093265,-89.491800,Crandalls Carryout & Catering,[],...,"Middleton town, Dane County, Wisconsin",5877.0,5831,5845,5943,6030,6086,6140,6179,6257
2,"{'Take-out': True, 'Good For': {'dessert': Fal...",SKLw05kEIlZcpTD5pqma8Q,"[Party & Event Planning, Asian Fusion, Event P...",Middleton,"2039 Allen Blvd\nMiddleton, WI 53562","{'Monday': {'close': '14:00', 'open': '11:00'}...",43.090642,-89.485169,Imperial Garden Chinese Restaurant,[],...,"Middleton city, Dane County, Wisconsin",17442.0,17558,17622,17903,18271,18522,18694,18955,19109
3,"{'Take-out': True, 'Good For': {'dessert': Fal...",SKLw05kEIlZcpTD5pqma8Q,"[Party & Event Planning, Asian Fusion, Event P...",Middleton,"2039 Allen Blvd\nMiddleton, WI 53562","{'Monday': {'close': '14:00', 'open': '11:00'}...",43.090642,-89.485169,Imperial Garden Chinese Restaurant,[],...,"Middleton town, Dane County, Wisconsin",5877.0,5831,5845,5943,6030,6086,6140,6179,6257
4,"{'Take-out': True, 'Wi-Fi': 'no', 'Good For': ...",77ESrCo7hQ96VpCWWdvoxg,"[Mexican, Restaurants]",Middleton,"6230 University Ave\nMiddleton, WI 53562","{'Monday': {'close': '21:00', 'open': '06:00'}...",43.091061,-89.487487,Mi Cocina,[],...,"Middleton city, Dane County, Wisconsin",17442.0,17558,17622,17903,18271,18522,18694,18955,19109


The `merge` function looks through the `'state'` and `'city'` columns of `yelp_df` and `census` and tries to match up rows that share values. When a match is found, the rows are combined. What happens when a match is not found? We can imagine four scenarios:  

1. We only keep rows from `yelp_df` and `census` if they match. Any rows from either table that have no match are discarded. This is called an _inner join_.  

2. We keep all rows from `yelp_df` and `census`, even if they have no match. In this case, when a row in `yelp_df` has no match in `census`, all the columns from `census` are merged in with null values. When a row in `census` has no match in `yelp_df`, all the columns from `yelp_df` are merged in with null values. This is called an _outer join_.

3. We privilege the `yelp_df` data. If a row in `yelp_df` has no match in `census`, we keep it and fill in the missing `census` columns as null values. If a row in `census` has no match in `yelp_df`, we discard it. This is called a _left join_.

4. We privilege the `census` data. This is called a _right join_.

The default behavior for Pandas is case #1, the _inner join_. This means if there are cities in `yelp_df` that we don't have matching `census` data for, they are dropped. Therefore, `merged_df` might be smaller than `yelp_df`.

In [ ]:
print(yelp_df.shape)
print(merged_df.shape)

There are a lot of cities in `yelp_df` that aren't in `census`! We might want to keep these rows, but we don't need any census data where there are no businesses. Then we should use a _left join_.

In [105]:
merged_df = yelp_df.merge(census, on=['state', 'city'], how='left')
print(yelp_df.shape)
print(merged_df.shape)

(37938, 15)
(39648, 27)


Sometimes we don't need to merge together the columns of separate data sets, but just need to add more rows. For example, the New York City subway system [releases data about how many customers enter and exit the station each week](http://web.mta.info/developers/turnstile.html). Each weekly data set has the same columns, so if we want multiple weeks of data, we just have to append one week to another.

In [106]:
nov18 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_171118.txt')
nov11 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_171111.txt')

In [108]:
nov18.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/11/2017,03:00:00,REGULAR,6402636,2166100
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/11/2017,07:00:00,REGULAR,6402645,2166112
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/11/2017,11:00:00,REGULAR,6402717,2166175
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/11/2017,15:00:00,REGULAR,6402857,2166230
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,11/11/2017,19:00:00,REGULAR,6403178,2166292


In [121]:
print(type(nov11['DATE'].unique()))

<class 'numpy.ndarray'>


In [109]:
nov = pd.concat([nov18, nov11])
nov['DATE'].unique()

array(['11/11/2017', '11/12/2017', '11/13/2017', '11/14/2017',
       '11/15/2017', '11/16/2017', '11/17/2017', '11/04/2017',
       '11/05/2017', '11/06/2017', '11/07/2017', '11/08/2017',
       '11/09/2017', '11/10/2017'], dtype=object)

We can also use `concat` to perform inner and outer joins based on index. For example, we can perform some data aggregation and then join the results onto the original DataFrame.

In [198]:
city_counts = yelp_df.groupby('city')['business_id'].count().rename('city_counts')
city_counts.reset_index()


,city,city_counts
0,Ahwatukee,8
1,Anthem,55
2,Apache Junction,80
3,Arcadia,1
4,Atlanta,1
5,Avondale,233
6,Black Canyon City,1
7,Bonnyrigg,2
8,Boulder City,22
9,Buckeye,71


In [203]:
yelp_df.get_values()[0]
city_counts.get_dtype_counts

<bound method NDFrame.get_dtype_counts of city
Ahwatukee                  8
Anthem                    55
Apache Junction           80
Arcadia                    1
Atlanta                    1
Avondale                 233
Black Canyon City          1
Bonnyrigg                  2
Boulder City              22
Buckeye                   71
C Las Vegas                1
Cambridge                  5
Carefree                  36
Casa Grande               87
Cave Creek               131
Centennial Hills           1
Central City Village       4
Central Henderson          1
Chandler                1468
Chandler-Gilbert           1
City of Edinburgh          1
Clark County               4
Coolidge                   8
Cottage Grove              9
Cramond                    2
Dalkeith                   3
Dane                       2
De Forest                  6
DeForest                   5
Deforest                   1
                        ... 
Sun City                  70
Sun City Anthem          

In [131]:
pd.concat([yelp_df.set_index('city'), city_counts], axis=1, join='inner').head()

,attributes,business_id,categories,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type,city_counts
city,,,,,,,,,,,,,,,
Phoenix,{'By Appointment Only': True},vcNAWiLM4dR7D2nwwJ7nCA,"[Doctors, Health & Medical]","4840 E Indian School Rd\nSte 101\nPhoenix, AZ ...","{'Tuesday': {'close': '17:00', 'open': '08:00'...",33.499313,-111.983758,"Eric Goldberg, MD",[],True,7,3.5,AZ,business,6715
De Forest,"{'Take-out': True, 'Good For': {'dessert': Fal...",JwUE5GmEO-sH1FuwJgKBlQ,[Restaurants],"6162 US Highway 51\nDe Forest, WI 53532",{},43.238893,-89.335844,Pine Cone Restaurant,[],True,26,4.0,WI,business,6
De Forest,"{'Take-out': True, 'Good For': {'dessert': Fal...",uGykseHzyS5xAMWoN6YUqA,"[American (Traditional), Restaurants]","505 W North St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '06:00'}...",43.252267,-89.353437,Deforest Family Restaurant,[],True,16,4.0,WI,business,6
De Forest,"{'Take-out': True, 'Wi-Fi': 'free', 'Takes Res...",LRKJF43s9-3jG9Lgx4zODg,"[Food, Ice Cream & Frozen Yogurt, Fast Food, R...","4910 County Rd V\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '10:30'}...",43.251045,-89.374983,Culver's,[],True,7,4.5,WI,business,6
De Forest,"{'Take-out': True, 'Has TV': False, 'Outdoor S...",RgDg-k9S5YD_BaxMckifkg,"[Chinese, Restaurants]","631 S Main St\nDe Forest, WI 53532","{'Monday': {'close': '22:00', 'open': '11:00'}...",43.240875,-89.343722,Chang Jiang Chinese Kitchen,[],True,3,4.0,WI,business,6


Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data.

## Working with time series
Pandas has a well-designed backend for inferring dates and times from strings and doing meaningful computations with them. 

In [204]:
pop_growth = pd.read_html('https://web.archive.org/web/20170127165708/https://www.census.gov/population/international/data/worldpop/table_population.php', attrs={'class': 'query_table'}, parse_dates=[0])[0]
pop_growth.dropna(inplace=True)
pop_growth.head()

,Year,Population,Annual Growth Rate (%),Annual Population Change
0,1950-01-01,2.557629e+09,1.459,37311223.0
1,1951-01-01,2.594940e+09,1.612,41832429.0
2,1952-01-01,2.636772e+09,1.717,45281083.0
3,1953-01-01,2.682053e+09,1.796,48174715.0
4,1954-01-01,2.730228e+09,1.900,51870839.0


By setting the `'Year'` column to the index, we can easily aggregate data by date using the `resample` method. The `resample` method allows us to decrease or increase the sampling frequency of our data. For instance, maybe instead of yearly data, we want to see average quantities for each decade.

In [205]:
pop_growth.set_index('Year', inplace=True)

In [206]:
pop_growth.resample('10AS').mean()

,Population,Annual Growth Rate (%),Annual Population Change
Year,,,
1950-01-01,2.765922e+09,1.7529,48537285.4
1960-01-01,3.321900e+09,2.0094,66969623.4
1970-01-01,4.047646e+09,1.8154,73179902.2
1980-01-01,4.813578e+09,1.7438,83875618.4
1990-01-01,5.651506e+09,1.4220,80123197.0
2000-01-01,6.429734e+09,1.2017,77213097.9
2010-01-01,7.206287e+09,1.0742,77318221.4
2020-01-01,7.949758e+09,0.8687,68934817.8
2030-01-01,8.587652e+09,0.6741,57810604.6


This kind of resampling is called _downsampling_, because we are decreasing the sampling frequency of the data. We can choose how to aggregate the data from each decade (e.g. `mean`). Options for aggregation include `mean`, `median`, `sum`, `last`, and `first`.

We can also _upsample_ data. In this case, we don't have data for each quarter, so we have to tell Pandas has to fill in the missing data.

In [208]:
pop_growth.resample('1Q').bfill().head()

,Population,Annual Growth Rate (%),Annual Population Change
Year,,,
1950-03-31,2.594940e+09,1.612,41832429.0
1950-06-30,2.594940e+09,1.612,41832429.0
1950-09-30,2.594940e+09,1.612,41832429.0
1950-12-31,2.594940e+09,1.612,41832429.0
1951-03-31,2.636772e+09,1.717,45281083.0


In [209]:
pop_growth.resample('1Q').ffill().head()

,Population,Annual Growth Rate (%),Annual Population Change
Year,,,
1950-03-31,2.557629e+09,1.459,37311223.0
1950-06-30,2.557629e+09,1.459,37311223.0
1950-09-30,2.557629e+09,1.459,37311223.0
1950-12-31,2.557629e+09,1.459,37311223.0
1951-03-31,2.594940e+09,1.612,41832429.0


Pandas' time series capabilities are built on the Pandas `Timestamp` class.

In [210]:
print(pd.Timestamp('January 8, 2017'))
print(pd.Timestamp('01/08/17 20:13'))
print(pd.Timestamp(1.4839*10**18))

2017-01-08 00:00:00
2017-01-08 20:13:00
2017-01-08 18:26:40


In [207]:
print(pd.Timestamp('Feb. 11 2016 2:30 am') - pd.Timestamp('2015-08-03 5:14 pm'))

191 days 09:16:00


In [212]:
from pandas.tseries.offsets import BDay, Day, BMonthEnd

print(pd.Timestamp('January 9, 2017') - Day(4))
print(pd.Timestamp('January 9, 2017') - BDay(4))
print(pd.Timestamp('January 9, 2017') + BMonthEnd(1))

2017-01-05 00:00:00
2017-01-03 00:00:00
2017-01-31 00:00:00


If we're entering time series data into a DataFrame it will often be useful to create a range of dates.

In [213]:
pd.date_range(start='1/8/2017', end='3/2/2017', freq='B')

DatetimeIndex(['2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18',
               '2017-01-19', '2017-01-20', '2017-01-23', '2017-01-24',
               '2017-01-25', '2017-01-26', '2017-01-27', '2017-01-30',
               '2017-01-31', '2017-02-01', '2017-02-02', '2017-02-03',
               '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09',
               '2017-02-10', '2017-02-13', '2017-02-14', '2017-02-15',
               '2017-02-16', '2017-02-17', '2017-02-20', '2017-02-21',
               '2017-02-22', '2017-02-23', '2017-02-24', '2017-02-27',
               '2017-02-28', '2017-03-01', '2017-03-02'],
              dtype='datetime64[ns]', freq='B')

The `Timestamp` class is compatible with Python's `datetime` module.

In [ ]:
import datetime

pd.Timestamp('May 1, 2017') - datetime.datetime(2017, 1, 8)

## Visualizing data with Pandas

Visualizing a data set is an important first step in drawing insights. We can easily pass data from Pandas to Matplotlib for visualizations, but Pandas also plugs into Matplotlib directly through methods like `plot` and `hist`.

In [ ]:
yelp_df['review_count'].apply(np.log).hist(bins=30)

In [ ]:
pop_growth['Annual Growth Rate (%)'].plot()

The [plotting functions take many parameters](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) for customizing the appearance of the output. Since they are essentially a wrapper around the Matplotlib functions, they also accept many of the Matplotlib parameters, not all of which are listed in the Pandas documentation. Pandas provides [a guide](https://pandas.pydata.org/pandas-docs/stable/visualization.html) to making various plots from DataFrames.

*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*